In [1]:
#Importar librerias
import pandas as pd

In [2]:
# Leer dataset original
data = pd.read_csv("casos_covid19_colombia.csv")

C:\Users\crist\AppData\Local\Temp\ipykernel_3448\4159600309.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("casos_covid19_colombia.csv")


In [3]:
data.shape

(6390971, 23)

# Preparación de los datos

In [4]:
# Selección de datos relevantes
clean_data = data[[
    "ID de caso",
    "Nombre departamento",
    "Nombre municipio",
    "Edad",
    "Unidad de medida de edad",
    "Sexo",
    "Tipo de contagio",
    "Estado",
    "Fecha de inicio de síntomas"]]

clean_data.shape

(6390971, 9)

In [5]:
# Limpieza de datos
# Estandarizar nombres de departamentos, municipios, valores categoricos de Sexo y valores de la columna Estado. 
clean_data.loc[:,"Nombre departamento"] = clean_data["Nombre departamento"].str.upper()
clean_data.loc[:,"Nombre municipio"] = clean_data["Nombre municipio"].str.upper()
clean_data.loc[:,"Sexo"] = clean_data["Sexo"].str.upper()
clean_data.loc[(clean_data["Estado"] == "leve") | (clean_data["Estado"] == "LEVE"),"Estado"] = "Leve"

# Remover valores N/A de la columna estado
clean_data = clean_data[~clean_data["Estado"].isna()]
clean_data.shape

# Remover valores invalidos de Unidad de medida de edad
clean_data = clean_data[clean_data["Unidad de medida de edad"]<=3]
clean_data.shape

(6349701, 9)

In [6]:
# Construcción de nuevos datos
# Categorización asintomaticos
clean_data.loc[~clean_data["Fecha de inicio de síntomas"].isna(),"Fecha de inicio de síntomas"] = 0
clean_data.loc[clean_data["Fecha de inicio de síntomas"].isna(),"Fecha de inicio de síntomas"] = 1
#clean_data["Fecha de inicio de síntomas"].value_counts()

# Modificación de edad basado en la unidad de medida de edad
clean_data.loc[clean_data["Unidad de medida de edad"] == 2,"Edad"] = clean_data["Edad"]/12.0
clean_data.loc[clean_data["Unidad de medida de edad"] == 3,"Edad"] = clean_data["Edad"]/365.0

# Creación de rango de edad
clean_data.loc[clean_data["Edad"] < 5,"rango_edad"] = "0 a 4 años"
clean_data.loc[(clean_data["Edad"] >= 5) & (clean_data["Edad"] < 12),"rango_edad"] = "5 a 11 años"
clean_data.loc[(clean_data["Edad"] >= 12) & (clean_data["Edad"] < 26),"rango_edad"] = "12 a 25 años"
clean_data.loc[(clean_data["Edad"] >= 26) & (clean_data["Edad"] < 41),"rango_edad"] = "26 a 40 años"
clean_data.loc[(clean_data["Edad"] >= 41) & (clean_data["Edad"] < 64),"rango_edad"] = "41 a 64 años"
clean_data.loc[clean_data["Edad"] >= 64,"rango_edad"] = "65 años y más"

C:\Users\crist\AppData\Local\Temp\ipykernel_3448\2389459221.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5        0.25       0.5        ... 0.91666667 0.5        0.16666667]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  clean_data.loc[clean_data["Unidad de medida de edad"] == 2,"Edad"] = clean_data["Edad"]/12.0
C:\Users\crist\AppData\Local\Temp\ipykernel_3448\2389459221.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0 a 4 años' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  clean_data.loc[clean_data["Edad"] < 5,"rango_edad"] = "0 a 4 años"


In [7]:
# Formato de datos 
# Remover columa de unidad de medida 
clean_data = clean_data.drop(columns=["Unidad de medida de edad"])
# Renombrar columnas a formato camel_case
clean_data = clean_data.rename(columns={
    "ID de caso": "id_caso",
    "Nombre departamento": "departamento",
    "Nombre municipio": "municipio",
    "Edad": "edad",
    "Sexo": "sexo",
    "Tipo de contagio": "tipo_contagio",
    "Estado": "estado",
    "Fecha de inicio de síntomas":"asintomatico"
    })

In [8]:
clean_data

,id_caso,departamento,municipio,edad,sexo,tipo_contagio,estado,asintomatico,rango_edad
0,1556979,VALLE,CALI,67.0,F,Comunitaria,Leve,0,65 años y más
1,1556980,VALLE,CALI,66.0,F,Comunitaria,Leve,0,65 años y más
2,1556981,VALLE,CALI,68.0,F,Comunitaria,Leve,0,65 años y más
3,1556982,VALLE,CALI,74.0,F,Comunitaria,Fallecido,0,65 años y más
4,1556983,VALLE,CALI,65.0,F,Comunitaria,Leve,0,65 años y más
...,...,...,...,...,...,...,...,...,...
6390966,4990088,VALLE,PALMIRA,37.0,M,Relacionado,Leve,0,26 a 40 años
6390967,4990091,VALLE,PALMIRA,36.0,M,Comunitaria,Leve,0,26 a 40 años
6390968,4990092,VALLE,JAMUNDI,39.0,F,Comunitaria,Leve,0,26 a 40 años
6390969,4990095,VALLE,JAMUNDI,43.0,F,Relacionado,Leve,0,41 a 64 años


In [9]:
# Exportar datos
clean_data.to_csv("covid19_clean_data.csv")